In [1]:
import pandas as pd
import os
from IPython.display import display, Markdown, HTML
%matplotlib inline
import tkinter as tk
from tkinter import filedialog, messagebox
from openpyxl import load_workbook
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

In [2]:
messagebox.showinfo('OnSSET', 'Open the input file with calibrated GIS data')
input_file = filedialog.askopenfilename()
df = pd.read_csv(input_file)

In [3]:
#df.sort_values(by=['Pop2018'], ascending=False, inplace=True)
#cumulative_pop = df['Pop2018'].cumsum()/df['Pop2018'].sum()

First, update the population and urbanization rate

In [4]:
years = [2025, 2030, 2040, 2050, 2060, 2070]
time_step = {2025: 7,
             2030: 5,
             2040: 10,
             2050: 10,
             2060: 10,
             2070: 10}

In [5]:
urbanization_rate = {2018: 0.2006, # Do not change the 2018 value, update the rest
                    2025: 0.263,
                    2030: 0.300,
                    2040: 0.375,
                    2050: 0.450,
                    2060: 0.525,
                    2070: 0.600}

population = {2018: 107535000, # Do not change the 2018 value, update the rest
              2025: 134208459.1,
              2030: 149774797.1,
              2040: 181383802.1,
              2050: 211590946.7,
              2060: 237724184.2,
              2070: 261168812}

household_size = {2018: 4.53, # Do not change the 2018 value, update the rest
                  2025: 4.26,
                  2030: 4.07,
                  2040: 3.69,
                  2050: 3.30,
                  2060: 3.00,
                  2070: 2.96}

In [6]:
for year in years:
    urban_growth = (urbanization_rate[year] * population[year])/(urbanization_rate[year-time_step[year]] * population[year-time_step[year]])
    rural_growth = ((1-urbanization_rate[year]) * population[year])/((1-urbanization_rate[year-time_step[year]]) * population[year-time_step[year]])
    df.loc[df['IsUrban'] == 2, 'Pop{}'.format(year)] = df['Pop{}'.format(year-time_step[year])] * urban_growth
    df.loc[df['IsUrban'] < 2, 'Pop{}'.format(year)] = df['Pop{}'.format(year-time_step[year])] * rural_growth
    df['NumPeoplePerHH' + '{}'.format(year)] = household_size[year] 

In [7]:
tier_1_urban = {2018: 19.1, 
                2025: 19.1,
                2030: 19.1,
                2040: 19.1,
                2050: 18,
                2060: 17,
                2070: 17}  # kWh/household/year

tier_2_urban = {2018: 229, 
                2025: 174,
                2030: 156,
                2040: 131,
                2050: 120,
                2060: 116,
                2070: 115}  # kWh/household/year

tier_3_urban = {2018: 619, 
                2025: 564,
                2030: 533,
                2040: 442,
                2050: 384,
                2060: 360,
                2070: 354}  # kWh/household/year

tier_4_urban = {2018: 1915, 
                2025: 1845,
                2030: 1741,
                2040: 1514,
                2050: 1415,
                2060: 1380,
                2070: 1371}  # kWh/household/year

tier_5_urban = {2018: 3785, 
                2025: 3497,
                2030: 3366,
                2040: 3007,
                2050: 2753,
                2060: 2622,
                2070: 2580}  # kWh/household/year

tier_1_rural = {2018: 5.9, 
                2025: 5.9,
                2030: 5.9,
                2040: 5.9,
                2050: 5.9,
                2060: 5.9,
                2070: 5.9}  # kWh/household/year

tier_2_rural = {2018: 181, 
                2025: 174,
                2030: 156,
                2040: 131,
                2050: 120,
                2060: 116,
                2070: 115}  # kWh/household/year

tier_3_rural = {2018: 572, 
                2025: 564,
                2030: 533,
                2040: 442,
                2050: 384,
                2060: 360,
                2070: 354}  # kWh/household/year

tier_4_rural = {2018: 1867, 
                2025: 1845,
                2030: 1741,
                2040: 1514,
                2050: 1415,
                2060: 1380,
                2070: 1371}  # kWh/household/year

tier_5_rural = {2018: 3737, 
                2025: 3497,
                2030: 3366,
                2040: 3007,
                2050: 2753,
                2060: 2622,
                2070: 2580}  # kWh/household/year

In [8]:
tier_split_2025 = {5: 0.03, # Share of population in Tier 5 
                   4: 0.14, # Share of population in Tier 5 or Tier 4
                   3: 0.14, # Share of population in Tier 5 or Tier 4 or Tier 3
                   2: 0.29, # Share of population in Tier 5 or Tier 4 or Tier 2
                   1: 1} # Share of population in Tier 5 or Tier 4 or Tier 2 or Tier 1 (Always = 1)

In [9]:
tier_split_2030 = {5: 0.03,
                   4: 0.20,
                   3: 0.21,
                   2: 0.37,
                   1: 1}

In [10]:
tier_split_2040 = {5: 0.05,
                   4: 0.38,
                   3: 0.45,
                   2: 0.63,
                   1: 1}

In [11]:
tier_split_2050 = {5: 0.07,
                   4: 0.49,
                   3: 0.90,
                   2: 1,
                   1: 1}

In [12]:
tier_split_2060 = {5: 0.11,
                   4: 0.59,
                   3: 1,
                   2: 1,
                   1: 1}

In [13]:
tier_split_2070 = {5: 0.16,
                   4: 0.67,
                   3: 1,
                   2: 1,
                   1: 1}

In [14]:
tier_splits = {2025: tier_split_2025,
               2030: tier_split_2030,
               2040: tier_split_2040,
               2050: tier_split_2050,
               2060: tier_split_2060,
               2070: tier_split_2070}

In [15]:
for year in years:
    
    df.sort_values(by=['Pop{}'.format(year)], ascending=False, inplace=True)
    cumulative_pop = df['Pop{}'.format(year)].cumsum()/df['Pop{}'.format(year)].sum()
    
    tier_split = tier_splits[year]
    
    tier_1_urb = tier_1_urban[year]
    tier_2_urb = tier_2_urban[year]
    tier_3_urb = tier_3_urban[year]
    tier_4_urb = tier_4_urban[year]
    tier_5_urb = tier_5_urban[year]
    
    tier_1_rur = tier_1_rural[year]
    tier_2_rur = tier_2_rural[year]
    tier_3_rur = tier_3_rural[year]
    tier_4_rur = tier_4_rural[year]
    tier_5_rur = tier_5_rural[year]
    
    # Tier 1
    df.loc[df['IsUrban'] == 2, 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_1_urb / household_size[year]
    df.loc[df['IsUrban'] < 2, 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_1_rur / household_size[year]
    df['Tier' + "{}".format(year)] = 1
    # Tier 2
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[2]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_2_urb / household_size[year]
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[2]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_2_rur / household_size[year]
    df.loc[cumulative_pop <= tier_split[2], 'Tier' + "{}".format(year)] = 2
    # Tier 3
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[3]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_3_urb / household_size[year]
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[3]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_3_rur / household_size[year]
    df.loc[cumulative_pop <= tier_split[3], 'Tier' + "{}".format(year)] = 3
    # Tier 4
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[4]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_4_urb / household_size[year]
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[4]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_4_rur / household_size[year]
    df.loc[cumulative_pop <= tier_split[4], 'Tier' + "{}".format(year)] = 4
    # Tier 5
    df.loc[(df['IsUrban'] == 2) & (cumulative_pop <= tier_split[5]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_5_urb / household_size[year]
    df.loc[(df['IsUrban'] < 2) & (cumulative_pop <= tier_split[5]), 'ResidentialDemandTierCustom' + "{}".format(year)] = tier_5_rur / household_size[year]
    df.loc[cumulative_pop <= tier_split[5], 'Tier' + "{}".format(year)] = 5

In [16]:
scenario_name = 'NewPoliciesEE_Modified_AS'

In [17]:
messagebox.showinfo('OnSSET', 'Browse to the folder where you want to save the outputs')
df.sort_index(inplace=True)
output_dir = filedialog.askdirectory()
output_dir_results = os.path.join(output_dir, '{}.csv'.format(scenario_name))
df.to_csv(output_dir_results, index=False)